#### In this step, we will try to detect the active features by stimulating the neural network and by transferring the inputs to the network and following the active paths, using the limited test data (```train.csv```) to recognize each of the numbers.

#### ```alpha``` HyperParameter filtering threshold

#### imports

In [ ]:
import numpy as np # matrix cals
import pandas as pd # data analys
from matplotlib import pyplot as plt # data Visualing
import json # for save json format
import seaborn as sns # data Visualing and Charts
import time # for save image file

In [ ]:
dataset = pd.read_csv('train.csv') # load Train DataSet

#### load wieghts

In [ ]:
with open('wgts\simpleMLPWghts.json', 'r') as openfile:
	json_object = json.load(openfile)

In [ ]:
json_object.keys()

In [ ]:
w1 = np.array(json_object['w1'])
w2 = np.array(json_object['w2'])
b1 = np.array(json_object['b1'])
b2 = np.array(json_object['b2'])

#### Layer Montioring

In [ ]:
def firstLayer(X,w1,b1):
    return ReLU(w1.dot(X) + b1)

In [ ]:
def ReLU(Z):
    return np.maximum(Z, 0)

In [ ]:
def softmax(Z):
    return  np.exp(Z) / sum(np.exp(Z))

In [ ]:
for i in range(10):
    values = np.array(dataset[:1000].query(f'label == {i}').T)   # peak vals
    value = values[1:] # choise image data
    value = value / 255.  # normalize
    plt.subplot(4,3,i+1)
    for j in range(len(value[0])):
        plt.plot(firstLayer(value[:,j,None],w1,b1), label=f'Img{j}')
plt.savefig(f"res\LayerMonitoring_Num{i}_{time.time() / 1000}.jpg")
plt.show()

In [ ]:
heat = []
firstLayers = []
for k in range(10):
    values = np.array(dataset[:1000].query(f'label == {k}').T)   # peak vals
    value = values[1:] # choise image data
    value = value / 255.  # normalize
    newList1 = []
    plt.subplot(4,3,k+1)
    heat.append([])
    heat[k] = 100 * [0]
    for j in range(len(value[0])):
        firstLayers = firstLayer(value[:,j,None],w1,b1)
    for i in range(len(firstLayers)):
        newList1.append([])
        for j in range(100):
            newList1[i].append(firstLayers[j][0])
            heat[k][j] += firstLayers[j][0]
    sns.heatmap(np.transpose(newList1))
plt.savefig(f"res\LayerMonitoringNodes_Num{k}_{time.time() / 1000}.jpg")
plt.show()

In [ ]:
heatSNS = sns.heatmap(np.transpose(heat), cmap=sns.cubehelix_palette(as_cmap=True))
plt.savefig(f"res\LayerMonitoringNodesAcumalator{time.time()}.jpg")
plt.show()

### filter Maxs [Average node Values for each epoch itereition]

In [ ]:
numberOfOperate = 0 # each Layer for each number
alpha = 0.4 # HyperParameter filtering threshold 

In [ ]:
datas = np.array(dataset[:1000].query(f'label == {numberOfOperate}'))

In [ ]:
dataBox = datas.T
numsX = dataBox[1:]
numsY = dataBox[0]
numsX = numsX / 255.

In [ ]:
firstLayers = []
for i in range(len(numsX[0])):
    firstLayers.append(ReLU(w1.dot(numsX[:,i,None])+ b1))

In [ ]:
newList1 = []
for i in range(len(firstLayers)):
    newList1.append([])
    for j in range(100):
        newList1[i].append(firstLayers[i][j][0])

In [ ]:
sns.heatmap(np.transpose(newList1))
plt.savefig(f"res\REV_LayerNodeMonitoring_{numberOfOperate}_alpha{alpha}_{time.time()}.jpg")
plt.show()

In [ ]:
FirstLayerMean = []
for i in range(100):
    FirstLayerMean.append(np.mean(np.transpose(newList1)[i]))

In [ ]:
sns.heatmap(np.transpose((FirstLayerMean,)))
plt.savefig(f"res\REV_LayerNodeMonitoringAverage_{numberOfOperate}_alpha{alpha}_{time.time()}_WithOutsoftmax.jpg")
plt.show()

In [ ]:

FirstLayerMean = softmax(FirstLayerMean)
sns.heatmap(np.transpose((FirstLayerMean,)))
plt.savefig(f"res\REV_LayerNodeMonitoringAverage_{numberOfOperate}_alpha{alpha}_{time.time()}_softmax.jpg")
plt.show()


In [ ]:
for i in range(len(FirstLayerMean)):
    if FirstLayerMean[i] < max(FirstLayerMean)*alpha : 
        FirstLayerMean[i] = 0

In [ ]:
sns.heatmap(np.transpose((FirstLayerMean,)))
plt.savefig(f"res\REV_LayerNodeMonitoringAverage_{numberOfOperate}\
    _alpha{alpha}_AfterFiltering_{time.time()}_WithOutsoftmax.jpg")
plt.show()

In [ ]:

sns.heatmap(np.transpose((FirstLayerMean,)))
plt.savefig(f"res\REV_LayerNodeMonitoringAverage_{numberOfOperate}\
    _alpha{alpha}_AfterFiltering_{time.time()}_softmax.jpg")
plt.show()


In [ ]:
rvW1AfterFiltering = w1.copy()

In [ ]:
numOfParams = 0
for i in range(rvW1AfterFiltering.shape[0]):
    if FirstLayerMean[i] == 0 :
        rvW1AfterFiltering[i] =  np.zeros((1,784)).tolist()[0]
    else :
        numOfParams += len(rvW1AfterFiltering[i])
print(f"Params count : {numOfParams}")

In [ ]:

sns.heatmap(rvW1AfterFiltering)
plt.savefig(f"res\REV_LayerNodeMonitoringAverage_ \
            {numberOfOperate}_alpha{alpha}_NewW1AfterFiltering_{time.time()}_WithOutsoftmax.jpg")
plt.show()


In [ ]:

sns.heatmap(rvW1AfterFiltering)
plt.savefig(f"res\REV_LayerNodeMonitoringAverage_{numberOfOperate}\
    _alpha{alpha}_NewW1AfterFiltering_{time.time()}_softmax.jpg")
plt.show()


### Comparison of accuracies

In [ ]:
truesAfter = 0
truesBefor = 0
for pr in range(len(numsY)):
    if numsY[pr] == np.argmax(softmax(w2.dot(ReLU(rvW1AfterFiltering.dot(numsX[:,pr ,None])+ b1)) + b2)):
        truesAfter += 1
    if numsY[pr] == np.argmax(softmax(w2.dot(ReLU(w1.dot(numsX[:,pr ,None])+ b1)) + b2)):
        truesBefor += 1
print("Befor Filter Acc : " , truesBefor / len(numsY))
print("After Filter Acc : " , truesAfter / len(numsY))

### Save Special Weights

- #### basic Save Weights to json file

In [ ]:
res = {
	"w1": rvW1AfterFiltering.tolist(),
	"w2": w2.tolist(),
    "b1": b1.tolist(),
	"b2": b2.tolist()
}
json_object = json.dumps(res, indent=4)
with open(f"wgts\simpleMLPSpecial{numberOfOperate}_alpha{alpha}_Wghts.json", "w") as outfile:
	outfile.write(json_object)

- ####  Vectorizer : Save Vector Weights to json file

In [ ]:
rvW1AfterFilteringDict = []
for i in range(rvW1AfterFiltering.shape[0]):
    if FirstLayerMean[i] != 0 :
        rvW1AfterFilteringDict.append((i , rvW1AfterFiltering[i].tolist()))

In [ ]:
res = {
	"w1": rvW1AfterFilteringDict.tolist(),
	"w2": w2.tolist(),
    "b1": b1.tolist(),
	"b2": b2.tolist()
}
json_object = json.dumps(res, indent=4)
with open(f"wgts\simpleMLPSpecial{numberOfOperate}_alpha{alpha}_WghtsVectorized.json", "w") as outfile:
	outfile.write(json_object)

### Integeration Weights

- #### simple weights

In [ ]:
w1 = np.zeros((100,784))
for i in range(10):
    with open(f"wgts\simpleMLPSpecial{i}_alpha{alpha}_Wghts.json", "r") as openfile:
        json_object = json.load(openfile)
    tmpw1 = np.array(json_object['w1'])
    for j in range(len(tmpw1)):
        if sum(w1[j]) == 0:
            w1[j] = tmpw1[j]

In [ ]:
sns.heatmap(w1)
plt.savefig(f"res\REV_LayerNodeMonitoringAverageFinalWghts_ \
            {numberOfOperate}_alpha{alpha}_NewW1AfterFiltering_{time.time()/1000}_Accms.jpg")
plt.show()

- #### vectorized weights

In [ ]:
w1 = np.zeros((100,784))
for i in range(10):
    with open(f"wgts\simpleMLPSpecial{i}_alpha{alpha}_WghtsVectorized.json", 'r') as openfile:
        json_object = json.load(openfile)
    for j in range(len(json_object['w1'])):
        if sum(w1[json_object['w1'][j][0]]) == 0:
            w1[json_object['w1'][j][0]] = json_object['w1'][j][1]

In [ ]:
sns.heatmap(w1)
plt.savefig(f"res\REV_LayerNodeMonitoringAverageFinalWghts_ \
            {numberOfOperate}_alpha{alpha}_NewW1AfterFiltering_{time.time()/1000}_Accms.jpg")
plt.show()

### Save comprased weights

- #### Save Simple comprased weights

In [ ]:
res = {
	"w1": w1.tolist(),
	"w2": w2.tolist(),
    "b1": b1.tolist(),
	"b2": b2.tolist()
}
json_object = json.dumps(res, indent=4)
with open(f"wgts\simpleMLPFinalComprased_alpha{alpha}.json", "w") as outfile:
	outfile.write(json_object)

- #### Save vectorized comprased weights

In [ ]:
rvW1AfterFilteringDict = []
for i in range(w1.shape[0]):
    if sum(w1[i]) != 0 :
        rvW1AfterFilteringDict.append((i , w1[i].tolist()))

In [ ]:
res = {
	"w1": rvW1AfterFilteringDict,
	"w2": w2.tolist(),
    "b1": b1.tolist(),
	"b2": b2.tolist()
}
json_object = json.dumps(res, indent=4)
with open(f"wgts\simpleMLPFinalComprasedWghtsVectorized_alpha{alpha}.json", "w") as outfile:
	outfile.write(json_object)

### accuracy for each number : 

In [ ]:
def get_accuracy(predictions, Y):
    return np.sum(predictions == Y) / Y.size

#### Load comprased weights

- ##### Load Simple comprased weights

In [ ]:
with open(f"wgts\simpleMLPFinalComprased_alpha{alpha}.json", "r") as openfile:
    json_object = json.load(openfile)
w1 = np.array(json_object['w1'])
w2 = np.array(json_object['w2'])
b1 = np.array(json_object['b1'])
b2 = np.array(json_object['b2'])

In [ ]:
for i in range(10):
    values = np.array(dataset[:1000].query(f'label == {i}').T)   # peak vals
    valueY = values[0]
    valueX = values[1:] # choise image data
    valueX = valueX / 255.  # normalize
    print(f"Accuracy {i} : ",get_accuracy(np.argmax(softmax(w2.dot(ReLU(w1.dot(valueX) + b1)) + b2),0), valueY))

- ##### Load Vectorized comprased weights

In [ ]:
with open(f"wgts\simpleMLPFinalComprasedWghtsVectorized_alpha{alpha}.json","r") as openfile:
    json_object = json.load(openfile)
tmpw1 = json_object['w1']
w2 = np.array(json_object['w2'])
b1 = np.array(json_object['b1'])
b2 = np.array(json_object['b2'])

In [ ]:
w1 = np.zeros((100,784))
for i in range(len(tmpw1)):
    w1[tmpw1[i][0]] = tmpw1[i][1]

In [ ]:
for i in range(10):
    values = np.array(dataset[:1000].query(f'label == {i}').T)   # peak vals
    valueY = values[0]
    valueX = values[1:] # choise image data
    valueX = valueX / 255.  # normalize
    print(f"Accuracy {i} : ",get_accuracy(np.argmax(softmax(w2.dot(ReLU(w1.dot(valueX) + b1)) + b2),0), valueY))